In [3]:
thresholds <- c(5, 6, 7)
print("hello")
for (thr in thresholds) {

  # SETTING PATH 
  path <- "/sharedFolder/Data/1_HTGTS/13_Revision1/pooled/"
  results <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", thr, "/")
  previousResults <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", thr, "/")
  riderPath <- "/home/Riderv2/"

  # FUNZIONI UTILI
  noExt <- function(x) tools::file_path_sans_ext(x)
  bnoExt <- function(x) basename(tools::file_path_sans_ext(x))
  fNumber <- function(x) {
    sapply(x, FUN = function(x) {
      strsplit(x,"__")[[1]][2]
    })
  }
  rRider <- function(x) {
    sapply(x, FUN = function(x) {
      strsplit(x,"-RIDER")[[1]][1]
    })
  }
  center_scale <- function(x) scale(x, scale = FALSE)

  # LIBRERIE
  library("argparser")
  library(gtools)
  library("GenomicRanges")
  library("ggVennDiagram")
  library(eulerr)
  options(warn = -1)

  # CARICAMENTO DATI
  Database <- read.table(paste(previousResults,"Database.csv",sep="/"), header=TRUE, sep=",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]

  a <- list.files(path, pattern=".bed", recursive=TRUE)
  a <- mixedsort(a)
  a2 <- sapply(a, FUN=function(x){strsplit(x,".bed")[[1]][1]})
  a2 <- sapply(a2, FUN=function(x){strsplit(x,"__")[[1]][2]})

  print("Ide ANALYSIS")
  dir.create(results, recursive=TRUE, showWarnings=FALSE)

  Database <- Database[grep("Ide", Database[,6]),]
  a <- list.files(path, pattern=".bed", recursive=TRUE)
  a <- mixedsort(a)
  a2 <- sapply(a, FUN=function(x){strsplit(x,".bed")[[1]][1]})
  a2 <- sapply(a2, FUN=function(x){strsplit(x,"__")[[1]][2]})
  a2 <- c(a2[grep("Ide",a2)], a2[grep("AID-KO",a2)], a2[grep("WT",a2)])
  a <- c(a[grep("Ide",a)], a[grep("AID-KO",a)], a[grep("WT",a)])

  annotate <- function(data1, data2){
    data2 <- read.table(data2, header=TRUE, sep=",")
    newNames <- c()
    for(i in rownames(data1)){
      if(unique(data2[which(data2$hotspotID==i),"annotation"]) == "NULL"){
        newNames <- append(newNames, paste(unique(data2[which(data2$hotspotID==i),c("Chromosome","ChrStart","ChrEnd")]), collapse="_"))
      } else if(length(strsplit(unique(data2[which(data2$hotspotID==i),"annotation"]), "_")[[1]]) > 3) {
        newNames <- append(newNames, paste(unique(data2[which(data2$hotspotID==i),c("Chromosome","ChrStart","ChrEnd")]), collapse="_"))
      } else {
        newNames <- append(newNames, unique(data2[which(data2$hotspotID==i), "annotation"]))
      }
    }
    rownames(data1) <- newNames
    return(data1)
  }
hgtgts_count=matrix(0,ncol=length(unique(Database[,7])),nrow=length(a2))
colnames(hgtgts_count)=sapply(unique(Database[,7]),FUN=toString)
rownames(hgtgts_count)=a2
hgtgts_rpm=hgtgts_count
hgtgts_fpkm=hgtgts_count
for(j in a){
    system(paste("echo",j))
temp=read.table(paste(path,"/",j,sep=""),sep="\t")
tempGG=GRanges(seqnames=as.matrix(temp[,1]),ranges=IRanges(start = as.numeric(temp[,2]),end=as.numeric(temp[,3])))    
    for(i in seq(nrow(Database))){
        system(paste("echo",i/nrow(Database)))
        gg=GRanges(seqnames=as.matrix(Database[i,1]),ranges=IRanges(start = as.numeric(Database[i,2]),end=as.numeric(Database[i,3])))    
        over=countOverlaps(gg,tempGG)
        name=as.vector(noExt(fNumber(j)))
        hgtgts_count[name,toString(Database[i,7])]= over
        hgtgts_rpm[name,toString(Database[i,7])]= ((over)/nrow(temp))*1000000
   }

}
write.table((t(hgtgts_count)),paste(results,"/","count_hgtgtsHeatmap_WITHEVERYTHING.csv",sep=""),col.names=NA,sep=",")
write.table((t(hgtgts_rpm)),paste(results,"/","rpm_hgtgtsHeatmap_PlusWITHEVERYTHING.csv",sep=""),col.names=NA,sep=",")
#Venn diagram plot. Check how many hotspot are in common
Database=read.table(paste(previousResults,"Database.csv",sep="/"),header=TRUE,sep=",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]

types=c("Ide")
conditions=c("DMSO","Taze","Vale")
for(j in types){
subBase=Database[grep(j,Database[,6]),]
x <- list()
count=1
for(i in conditions){
x[[count]]=subBase[grep(i,subBase[,6]),7]
count=count+1
}
names(x)=conditions
system("echo yeah")
}
jjj=(unlist(x))
j2=x 
fit1 <- euler(c("DMSO" = length(setdiff(setdiff(j2$DMSO,j2$Taze),j2$Vale)), "Taze" = length(setdiff(setdiff(j2$Taze,j2$DMSO),j2$Vale)), "Vale" = length(setdiff(setdiff(j2$Vale,j2$Taze),j2$DMSO)),"DMSO&Taze&Vale"=length(intersect(intersect(j2$DMSO,j2$Taze),j2$Vale)),"DMSO&Taze"=length(setdiff(intersect(j2$DMSO,j2$Taze),j2$Vale)),"DMSO&Vale"=length(setdiff(intersect(j2$DMSO,j2$Vale),j2$Taze)),"Taze&Vale"=length(setdiff(intersect(j2$Taze,j2$Vale),j2$DMSO)) ))
DMSOSpec=setdiff(setdiff(x$DMSO,x$Vale),x$Taze)
TazeSpec=setdiff(setdiff(x$Taze,x$DMSO),x$Vale)
ValeSpec=setdiff(setdiff(x$Vale,x$DMSO),x$Taze)
TazVale=setdiff(intersect(x$Taze,x$Vale),x$DMSO)
TazDMSO=setdiff(intersect(x$Taze,x$DMSO),x$Vale)
DMSOVale=setdiff(intersect(x$ Vale,x$DMSO),x$Taze)
common=intersect(intersect(x$ Vale,x$DMSO),x$Taze)
fit1 <- euler(c("DMSO" = length(setdiff(j2$DMSO,j2$Taze)), "Taze" = length(setdiff(j2$Taze,j2$DMSO)),"DMSO&Taze"=length(intersect(j2$DMSO,j2$Taze)) ))
fit1 <- euler(c("Taze" = length(setdiff(j2$Taze,j2$Vale)), "Vale" = length(setdiff(j2$Vale,j2$Taze)),"Taze&Vale"=length(intersect(j2$Vale,j2$Taze)) ))
fit1 <- euler(c("DMSO" = length(setdiff(j2$DMSO,j2$Vale)), "Vale" = length(setdiff(j2$Vale,j2$DMSO)),"DMSO&Vale"=length(intersect(j2$DMSO,j2$Vale)) ))
#GENERATING RPM PLOTS RAW
mj=list()
count=1
dir.create(paste(results,"/rpmPlot/",sep=""))
a=t(hgtgts_rpm)
temp=a[sapply(setdiff(j2$DMSO,j2$Taze),toString),]
if(length(temp)>2){
temp=temp[sort(temp[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp2=a[sapply(intersect(j2$DMSO,j2$Taze),toString),]
if(length(temp2)>2){
temp2=temp2[sort(temp2[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp3=a[sapply(setdiff(j2$Taze,j2$DMSO),toString),]
if(length(temp3)>2){
temp3=temp3[sort(temp3[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}

mj[[count]]=temp
count=count+1
mj[[count]]=temp2

count=count+1
mj[[count]]=temp3



count=count+1
Idetaze=rbind(temp,temp2,temp3)
filteredList=rownames(read.table(paste(results,"/rpmPlot/filtered/filtered_Ide_DMSOTaze.csv",sep=""),header=TRUE,row.names=1,sep=","))
write.table(annotate(Idetaze,paste(previousResults,"Database_annotated.csv",sep="/")),paste(results,"/rpmPlot/Ide_DMSOTaze_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)
Idetaze=Idetaze[filteredList,]
write.table(annotate(Idetaze,paste(previousResults,"Database_annotated.csv",sep="/")),paste(results,"/rpmPlot/filtered/filtered_Ide_DMSOTaze_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)





temp=a[sapply(setdiff(j2$DMSO,j2$Vale),toString),]
if (is.null(dim(temp))) {
    temp=t(as.matrix(temp[sort(temp,index.return=TRUE,decreasing=TRUE)$ix]))
    rownames(temp)=sapply(setdiff(j2$DMSO,j2$Vale),toString)
}else if(length(temp)>2){
temp=temp[sort(temp[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp2=a[sapply(intersect(j2$DMSO,j2$Vale),toString),]
if(length(temp2)>2){
temp2=temp2[sort(temp2[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp3=a[sapply(setdiff(j2$Vale,j2$DMSO),toString),]
if(length(temp3)>2){
temp3=temp3[sort(temp3[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}


mj[[count]]=temp
count=count+1
mj[[count]]=temp2

count=count+1
mj[[count]]=temp3

count=count+1
write.table(rbind(temp,temp2,temp3),paste(results,"/rpmPlot/Ide_DMSOVale_PlusAIDKO.csv",sep=""),col.names=NA,sep=",",quote=TRUE)
Idevale=rbind(temp,temp2,temp3)
filteredList=rownames(read.table(paste(results,"/rpmPlot/filtered/filtered_Ide_DMSOVale.csv",sep=""),header=TRUE,row.names=1,sep=","))
write.table(annotate(Idevale,paste(previousResults,"Database_annotated.csv",sep="/")),paste(results,"/rpmPlot/Ide_DMSOVale_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)
Idevale=Idevale[filteredList,]
write.table(annotate(Idevale,paste(previousResults,"Database_annotated.csv",sep="/")),paste(results,"/rpmPlot/filtered/filtered_Ide_DMSOVale_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)

}

[1] "hello"
[1] "Ide ANALYSIS"
[1] "Ide ANALYSIS"
[1] "Ide ANALYSIS"
